In [1]:
import re
import requests
import pandas as pd

Грузим информацию (я скопировал файл к себе на компьютер)

In [2]:
df = pd.read_csv('rest_data.csv')

Методом extract достаем регуляркой все между ", " и "," (улицу)

In [3]:
df['address_street'] = df['address'].str.extract(', (.*?),')

У нас там получилось около 50 пустых улиц потому что в адресе не был написан город, добавляем город

In [4]:
df.loc[df['address_street'].isna(), 'address'] = 'город Москва, ' + df['address']

Выдергиваем улицу еще раз, теперь все нормально

In [5]:
df['address_street'] = df['address'].str.extract(', (.*?),')

ТОП 10 улиц минус 'город Зеленоград' и 'поселение Сосенское' потому что они из новой Москвы

In [6]:
top10_street = df[~df['address_street'].isin(['город Зеленоград','поселение Сосенское'])].\
               pivot_table(index='address_street',values='id', aggfunc='count').\
               sort_values('id', ascending=False).head(10)

In [9]:
street_one = df[~df['address_street'].isin(['город Зеленоград','поселение Сосенское'])].\
               pivot_table(index='address_street',values='id', aggfunc='count')
street_one = street_one[street_one['id'] == 1]
street_one

,id
address_street,
1-й Балтийский переулок,1
1-й Басманный переулок,1
1-й Ботанический проезд,1
1-й Вешняковский проезд,1
1-й Голутвинский переулок,1
...,...
улица Шеногина,1
улица Шумилова,1
улица Шумкина,1


Создаем датафрейм для запроса (улица в адресе должна быть из топ10 улиц)

In [11]:
data_for_request = df[df['address_street'].isin(street_one.index)][['id','address']]
data_for_request

,id,address
19,153644,"город Москва, Филипповский переулок, дом 15/5"
61,21214,"город Москва, Бабаевская улица, дом 3"
69,22139,"город Москва, 1-й Балтийский переулок, дом 3/25"
81,25833,"город Москва, улица Демьяна Бедного, дом 4"
131,150881,"город Москва, Будайская улица, дом 3"
...,...,...
14227,208805,"город Москва, 3-й Угрешский проезд, дом 8, стр..."
14440,211368,"город Москва, улица Линии Октябрьской Железной..."
14462,216767,"город Москва, 80-й километр Московской Кольцев..."
14704,214324,"город Москва, 2-й Кадашёвский переулок, дом 16..."


Создаем пустой массив под координаты

In [16]:
coord_list = []

Для каждого адреса из датафрейма для запроса делаем запрос к АПИ яндекса, результаты запроса кладем в переменную и из нее регуляркой достаем координаты из тега `<pos>`, можно было распарсить супом как Артем советовал но мне было лень :)

In [17]:
for address in data_for_request['address']:
    URL_for_coord = 'https://geocode-maps.yandex.ru/1.x/?apikey=ключ&geocode=' + address
    data_for_coord = requests.get(URL_for_coord).text
    try:
        coord = re.search('<pos>(.*?)<', data_for_coord).group(1)
    except:
        coord = ''    
    coord_list.append(coord)

Записываем полученный массив координат в датафрейм для запроса (он у нас по порядку идет так что все встанет на свои места)

In [19]:
data_for_request['coord'] = coord_list
data_for_request.to_csv('coord.csv')

Создаем массив для районов

In [20]:
district_list = []

Делаем запрос аналогичный предыдущему только передаем не адрес а полученные координаты, в запросе указываем &kind=district для того чтобы апи вернул нам адрес с точностью до района, он будет в ответе между тегами `<formatted>` отсюда регуляркой и достаем

In [21]:
for coord in data_for_request['coord']:
    URL_for_district = 'https://geocode-maps.yandex.ru/1.x/?apikey=63551c59-5cab-4913-a44f-c78b79cc27d8&kind=district&geocode=' + coord
    data_for_district = requests.get(URL_for_district).text
    try:
        district = re.search('<formatted>(.*?)<', data_for_district).group(1)
    except:
        district = ''
    district_list.append(district)

Записываем в датафрейм районы

In [22]:
data_for_request['district'] = district_list
data_for_request

,id,address,coord,district
19,153644,"город Москва, Филипповский переулок, дом 15/5",37.598573 55.75058,"Россия, Москва, Центральный административный о..."
61,21214,"город Москва, Бабаевская улица, дом 3",37.689069 55.790595,"Россия, Москва, Восточный административный окр..."
69,22139,"город Москва, 1-й Балтийский переулок, дом 3/25",37.518785 55.810471,"Россия, Москва, Северный административный окру..."
81,25833,"город Москва, улица Демьяна Бедного, дом 4",37.48693 55.770401,"Россия, Москва, Северо-Западный административн..."
131,150881,"город Москва, Будайская улица, дом 3",37.665156 55.831903,"Россия, Москва, Северо-Восточный административ..."
...,...,...,...,...
14227,208805,"город Москва, 3-й Угрешский проезд, дом 8, стр...",37.699822 55.712315,"Россия, Москва, Юго-Восточный административный..."
14440,211368,"город Москва, улица Линии Октябрьской Железной...",37.572082 55.838819,"Россия, Москва, Северный административный окру..."
14462,216767,"город Москва, 80-й километр Московской Кольцев...",0 0,
14704,214324,"город Москва, 2-й Кадашёвский переулок, дом 16...",37.621902 55.743745,"Россия, Москва, Центральный административный о..."


Сохраняем к себе на компьютер

In [55]:
data_for_request.to_csv('district.csv')